In [102]:
import cv2
import numpy as np


In [108]:
def getIDCornerMap(gray):
        
    aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_100)
    parameters = cv2.aruco.DetectorParameters()
    
    detector = cv2.aruco.ArucoDetector(aruco_dict, parameters)
    corners, ids, _ = detector.detectMarkers(gray)
    if ids is None or len(ids) < 4:
        raise ValueError("Not all 4 ArUco markers were detected.")
    
    # Create a dictionary to map ID to its corner
    id_corner_map = {}
    for id, corner in zip(ids, corners):
        id_corner_map[id[0]] = corner
    
    if (48 in id_corner_map):
        nc=5
    elif (49 in id_corner_map):
        nc=4
    return id_corner_map,nc

def get_center(corner):
    return corner[0].mean(axis=0)
def get_centeroid(array):
    return np.array(array).mean(axis=0)
def getROI(id_corner_map):
    '''returns the orderded center of the four corners locations
    and metadata:number of choices per questions
    '''
    if (48 in id_corner_map):
        ordered_ids = [30, 10, 48, 34]  # TL, TR, BR, BL
    elif (49 in id_corner_map):
        ordered_ids = [30, 10, 49, 34]  # TL, TR, BR, BL
        
    ordered_pts = [get_center(id_corner_map[id]) for id in ordered_ids]
    
    roi= np.array(ordered_pts, dtype='float32')
    return roi

def getWarpedImage(src_pts,image):
    width, height = int(image.shape[0]*.764),image.shape[0]
    dst_pts = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]
    ], dtype='float32')
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, M, (width, height))
    return warped

def getLargestContour(image,drawContour=False):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    edged = cv2.Canny(gray, 30, 200) 
    

    contours, hierarchy = cv2.findContours(edged, 
    	cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
    largest_contour = max(contours, key=cv2.contourArea)
    
    #draw for debugging
    if drawContour:
        cv2.drawContours(image, largest_contour, -1, (0, 255, 0), 3) 
        cv2.imshow('largest contour', image) 
        cv2.waitKey(0) 
        cv2.destroyAllWindows() 
    return largest_contour

def getBubbleContours(binary_image,drawContours=False):
   
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, 
                                   cv2.CHAIN_APPROX_NONE)
    bubble_contours = []
    for c in contours:
        area = cv2.contourArea(c)
        x, y, w, h = cv2.boundingRect(c)
        aspect_ratio = w / float(h)
        
        if area > 50 and 0.9 < aspect_ratio < 1.1:
            bubble_contours.append(c)
    
    #sorting
    bubble_contours.sort(key=lambda b:get_centeroid(b)[0][1])
    
    # For sorting 15 elements at a time
    for i in range(10):  # Assuming you have 10 groups of 15 elements
        # Calculate the start and end indices for the current group
        start_idx = i * 15
        end_idx = (i + 1) * 15  # This gives you exactly 15 elements
        
        # Sort this slice and assign it back using sorted() which returns a new sorted list
        bubble_contours[start_idx:end_idx] = sorted(bubble_contours[start_idx:end_idx],key=lambda b: get_centeroid(b)[0][0])
    
    if drawContours:
        output = cropped.copy()
        cv2.drawContours(output, bubble_contours, -1, (0, 255, 0), 2)
        output_resized = cv2.resize(output, (800, int(output.shape[0] * 800 / output.shape[1])))
        cv2.imshow('Detected Bubbles', output_resized)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    return bubble_contours

def getFilledBubblesMatrix(bubble_contours,binary_image):
    extracted_matrix=[]
    for c in bubble_contours:
        # Step 1: Create a black mask
        mask = np.zeros(binary_image.shape, dtype="uint8")
    
        # Step 2: Draw the bubble contour filled with white on the mask
        cv2.drawContours(mask, [c], -1, 255, -1)
    
        # Step 3: Apply the mask to binary_image image
        masked_bubble = cv2.bitwise_and(binary_image, binary_image, mask=mask)
    
        # Step 4: Calculate number of white pixels
        total = cv2.countNonZero(mask)
        filled = cv2.countNonZero(masked_bubble)
    
        fill_ratio = filled / total
        #print(fill_ratio)
        if fill_ratio > 0.5:  # You can tweak 0.5 based on real data
            extracted_matrix.append(1)
        else:
            extracted_matrix.append(0)
    
    #np.array(extracted_matrix).reshape(10,15)
    return np.array(extracted_matrix).reshape(-1,5)

In [ ]:
class BubbleSheetGrader:
    def __init__(self,image_path):
        self.image_path=image_path
        self.nc=None
        self.image=None
        self.gray=None
        self.warped=None
        self.cropped=None
        self.thresh=None
        self.bubble_contours=None
        self.result=None
    def load_image(self):
        self.image = cv2.imread(self.image_path)
        self.gray = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)

    def warp_paper(self):
        id_corner_map,self.nc=getIDCornerMap(self.gray)
        self.warped=getWarpedImage(src_pts = getROI(id_corner_map),image=self.image)
    def preprocess_cropped(self):
        x, y, w, h = cv2.boundingRect(getLargestContour(warped))
        self.cropped = self.warped[y+10:y+h-10, x+10:x+w-10]
        
        cropped_gray = cv2.cvtColor(self.cropped, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(cropped_gray, (3, 3), 0)
        #return binary image based on adaptive thresholding
        self.thresh = cv2.adaptiveThreshold(blurred, 255, 
                                       cv2.ADAPTIVE_THRESH_MEAN_C, 
                                       cv2.THRESH_BINARY_INV, 49, 20)
    def extract_bubble_contours(self):
        self.bubble_contours=getBubbleContours(self.thresh,False)

    def extractAndEvaluateAnswerMatrix():
        answer_matrix=getFilledBubblesMatrix(bubble_contours,thresh)
        self.result =np.all(sam==mam,axis=1).sum()
    def run_pipeline(self):
        self.load_image()
        self.warp_paper()
        self.preprocess_cropped()
        self.extract_bubble_contours()
        self.extractAndEvaluateAnswerMatrix()

In [109]:
image = cv2.imread('../Downloads/studentanswer.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [110]:
id_corner_map,nc=getIDCornerMap(gray)

In [111]:
warped=getWarpedImage(src_pts = getROI(id_corner_map),image=image)

In [113]:
x, y, w, h = cv2.boundingRect(getLargestContour(warped))
cropped = warped[y+10:y+h-10, x+10:x+w-10]

cropped_gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(cropped_gray, (3, 3), 0)
#return binary image based on adaptive thresholding
thresh = cv2.adaptiveThreshold(blurred, 255, 
                               cv2.ADAPTIVE_THRESH_MEAN_C, 
                               cv2.THRESH_BINARY_INV, 49, 20)

bubble_contours=getBubbleContours(thresh,True)


In [114]:
sam=getFilledBubblesMatrix(bubble_contours,thresh)

In [115]:
sam==mam
np.all(sam==mam,axis=1).sum()

16